<a href="https://colab.research.google.com/github/khlose/homl_ageron/blob/master/homl_cp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import tarfile
from six.moves import urllib
import pandas as pd

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets","housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL,housing_path=HOUSING_PATH):
  if not os.path.isdir(housing_path):
    os.makedirs(housing_path)
  
  tgz_path = os.path.join(housing_path,"housing.tgz")
  
  urllib.request.urlretrieve(housing_url,tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()
  
  
  

def load_housing_data(housing_path=HOUSING_PATH):
  csv_path = os.path.join(housing_path,"housing.csv")
  return pd.read_csv(csv_path)




fetch_housing_data()
housing = load_housing_data()


#housing.head()

#housing["ocean_proximity"].value_counts()
#housing.describe()

import matplotlib.pyplot as plt
#housing.hist(bins=50,figsize=(20,15))


import numpy as np
from zlib import crc32

def test_set_check(identifier, test_ratio):
  return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test(data,test_ratio,id_column):
  
  ids = data[id_column]
  in_test_set = ids.apply(lambda id_:test_set_check(id_,test_ratio))
  
  return data.loc[~in_test_set], data.loc[in_test_set]





#housing_with_id = housing.reset_index()
#housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
#train,test = split_train_test(housing_with_id,0.2,"index")

from sklearn.model_selection import train_test_split

train_set,test_set = train_test_split(housing,test_size=0.2,random_state=42)



housing["income_cat"] = np.ceil(housing["median_income"]/1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0,inplace=True)

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state=42)

for train_index,test_index in split.split(housing,housing["income_cat"]):
  strat_train_data = housing.loc[train_index]
  strat_test_data = housing.loc[test_index]

  
print(strat_train_data.count())  


#basically loop twice, first loop is start_train_data loop
#and second is strat_test_data loop
for set_ in (strat_train_data,strat_test_data):
  print(set_.count())
  
  set_.drop("income_cat",axis=1,inplace=True)

In [0]:
housing = strat_train_data.copy()

corr_matrix = housing.corr()

#corr_matrix["median_house_value"].sort_values()

from pandas.plotting import scatter_matrix

#attributes = ["median_house_value","median_income","total_rooms","housing_median_age"]

#scatter_matrix(housing[attributes])

housing.plot(kind="scatter",x="median_income",y="median_house_value",alpha=0.1)

In [0]:
housing["room_per_household"] = housing["total_rooms"] / housing["households"]

housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]

housing["populations_per_household"] = housing["population"]/housing["households"]

corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values()

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

def imputeData(data):
  
  ret_data = data.copy()
  
  object_list = list(data.select_dtypes(include=['object']).columns)
  ret_data.drop(columns=object_list,axis=1,inplace=True)

  imputer = SimpleImputer(strategy="median")
  imputer.fit(ret_data)
  
  X = imputer.transform(ret_data)

  ret_data = pd.DataFrame(X,columns=ret_data.columns)
  
  #attach back the column with object
  
  wo_object =ret_data.copy()
  
  for column in object_list:
    ret_data[column] = data[column]
    
  #return dataframe without object column and a frame WITH object column
  return wo_object,ret_data

def objectDataProc(data):
  ret_data = data.copy()
  encoder = OneHotEncoder(handle_unknown='ignore')
  encoder.fit(ret_data)
  
  housing_cat_1hot = encoder.transfor(ret_data)
  print(housing_cat_1hot.describe())
  
  return housing_cat_1hot
  

In [13]:
#reset
housing = strat_train_data.copy()

housing_pred = strat_train_data.drop("median_house_value",axis=1)
housing_label = strat_train_data["median_house_value"].copy()


housing_tr_nObj, housing_tr_reg = imputeData(housing_pred)

print(housing_tr_reg["ocean_proximity"][13])

#print(housing_tr_reg["ocean_proximity"].isna())

housing_cat = housing["ocean_proximity"]
housing_cat_encoded,housing_categories = housing_cat.factorize(na_sentinel=-2)

print(housing_categories)

print(housing_cat_encoded[13])

unique, counts = np.unique(housing_cat_encoded, return_counts=True)

dict(zip(unique, counts))
#data_1hot = objectDataProc(housing_tr_reg)

nan
Index(['<1H OCEAN', 'NEAR OCEAN', 'INLAND', 'NEAR BAY', 'ISLAND'], dtype='object')
2


{0: 7276, 1: 2124, 2: 5263, 3: 1847, 4: 2}